# Zero Dependencies

## Setup

In [3]:
import json

In [1]:
dependency_chain_path = '../data/dependency_chain10.json'
most_depended_upon_path = '../data/most_depended_upon500.json'
registry_entries = '../data/registry_entries.json'

In [6]:
registry_map = {}

for p in json.load(open(registry_entries)):
    p_name = p['name']
    p_data = {}
    p_data['versions'] = p['versions']
    registry_map[p_name] = p_data

## 500 Most depended upon libraries from Libraries.io

In [49]:
#the libraries.io info for each package
most_depended_upon = json.load(open(most_depended_upon_path))

print(len(most_depended_upon))

500


## For the most depended upon libraries, how many have zero dependencies? What are the packages at the end of the dependency chain (zero dependencies)?

1. Get the most popular packages from `libraries.io` (rank: dependant count).

2. Get the dependency chain using the NPM registry for the latest version of each package. We ignore dev dependencies (only used in development) and peer dependencies (which may not be directly relied upon). Libraries.io only gives a single combined number, so any packages that with test suites for example will have dependencies.

In [8]:
dependency_chain = json.load(open(dependency_chain_path))

root_zeros = []
zeros = []
parents = []
nodes = []
def walk_chain(chain):
    for d in chain:
        if isinstance(chain, dict):
            d = chain[d]
        parents.append(d)
        if isinstance(d, str):
            #is end of chain
            zeros.append(d)
            if len(parents) == 1:
                root_zeros.append(d)
        else:
            walk_chain(d)
        parents.pop()


walk_chain(dependency_chain)
unique_zeros = list(dict.fromkeys(zeros))


print("Zero dependencies in chain:", len(zeros))
print("Unique zero dependencies in chain:", len(unique_zeros))
print("Zero dependency roots:", len(root_zeros), "/", len(dependency_chain), '(', len(root_zeros)/len(dependency_chain), ')')

# print("\nPopular zeros:", json.dumps(root_zeros, indent=1))
json.dump(root_zeros, open('../data/zero500.json', 'w'), indent=1)

# print("\nFinal nodes:", json.dumps(zeros, indent=1))
json.dump(zeros, open('../data/zeroOnChain500.json', 'w'), indent=1)


Zero dependencies in chain: 304
Unique zero dependencies in chain: 115
Zero dependency roots: 5 / 10 ( 0.5 )


## How many used to have dependencies?

In [37]:
reduced = {}

for p in root_zeros:
    has_dependencies = False
    versions = registry_map[p]['versions']
    v_data = {}
    for v in versions:
        dependencies = {}
        if 'dependencies' in versions[v]:
            dependencies = versions[v]['dependencies']
            if dependencies :
                has_dependencies = True
                dependencies = list(dependencies.keys())
                v_data[v] = dependencies
    if has_dependencies:
        reduced[p] = v_data

print('packages that reduced dependencies:', list(reduced.keys()))

packages that reduced dependencies: ['typescript', 'prettier']
